In [1]:
import sys
sys.path.append('..')

In [2]:
from training.generators.FileImageGenerator import create_image_lists, get_generators

Using TensorFlow backend.


In [3]:
IMG_DIR = '..\\data\\atari_v1\\screens\\mspacman'
VAL_PCT = 25
VAL_FORMAT = 'png'

In [4]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

ValueError: Image directory ..\data\atari_v1\screens\mspacman not found.

In [ ]:
IMG_SIZE=(160, 210, 3)
BATCH_SIZE = 5
EPIS_LEN = 6
EPIS_SHIFT = 2

In [ ]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode='episode', 
    episode_len=EPIS_LEN, 
    episode_shift=EPIS_SHIFT
)

In [ ]:
import tensorflow as tf

In [ ]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((BATCH_SIZE, EPIS_LEN, ) + IMG_SIZE), 
                   tf.TensorShape((BATCH_SIZE, EPIS_LEN, ) + IMG_SIZE)
                  )
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen,     
    output_types=(tf.float32, tf.float32) ,
    output_shapes=(tf.TensorShape((BATCH_SIZE, EPIS_LEN, ) + IMG_SIZE), 
                   tf.TensorShape((BATCH_SIZE, EPIS_LEN, ) + IMG_SIZE)
                  )
)


In [ ]:
dataset_name = 'atari_v1'

In [ ]:
intermediate_dim = 10
input_shape= (EPIS_LEN, )+IMG_SIZE # image shape
LATENT_DIM = 5

In [ ]:
input_shape

In [ ]:
tdConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(**kwds))

In [ ]:
tdDeConv = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2DTranspose(**kwds))

In [ ]:
tdDense = lambda **kwds: tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(**kwds))

In [ ]:
import numpy as np

In [ ]:
c = 3

In [ ]:
enc_mean_lays = [
    tdConv(filters=c, kernel_size=3, strides=(2, 2), activation='relu'),
    tdConv(filters=2*c, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(LATENT_DIM)
]

enc_var_lays = [
    tdConv(filters=c, kernel_size=3, strides=(2, 2), activation='relu'),
    tdConv(filters=2*c, kernel_size=3, strides=(2, 2), activation='relu'),
    tf.keras.layers.Flatten(),
    # No activation
    tf.keras.layers.Dense(LATENT_DIM)
]



dec_lays = [
    tf.keras.layers.Dense(units=c**3, activation=tf.nn.relu),
    tf.keras.layers.Reshape(target_shape=(1, c, c, c)),
    tdDeConv(filters=2*c, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    tdDeConv(filters=c, kernel_size=3, strides=(2, 2), padding="SAME", activation='relu'),
    
    # No activation
    tdDeConv(filters=1, kernel_size=3, strides=(1, 1), padding="SAME"),    
]

In [ ]:
model_name = 'Atari_VAE_tconv'

In [ ]:
recoding_dir='..\\recoding\\'+model_name

In [ ]:
import os
os.path.join(recoding_dir, model_name)

In [ ]:
from training.autoencoders_conditional.Cond_VAE import CondVAE as AE

In [ ]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'input_shape':input_shape,
        'output_shape':LATENT_DIM,
        'layers': enc_mean_lays
    }
    ,
    
     {
        'name': 'inference_logvariance', 
        'input_shape':input_shape,
        'output_shape':LATENT_DIM,
        'layers':enc_var_lays
    }
    ,
    
        {
        'name': 'decoder', 
        'input_shape':LATENT_DIM,
        'output_shape':input_shape,
        'layers':dec_lays
    }
]

In [ ]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [ ]:
_restore

In [ ]:
ae = AE( 
    model_name=model_name, 
    input_shape=input_shape, 
    latent_dim=LATENT_DIM, 
    variables_params=variables_params, 
    restore=None
    )

In [ ]:
import numpy as np
n_generated = 36
#random_latent = np.random.normal(size=[n_generated, latent_dim])
#np.save(file='random_latent.npy', arr=random_latent)
#random_latent = tf.constant(np.load(file='..\\data\\random_latent.npy'))

In [ ]:
random_latent = tf.constant(np.random.normal(size=[n_generated, LATENT_DIM]))
random_latent.shape

In [ ]:
ae.fit(
    train_dataset=train_ds, 
    test_dataset=test_ds,
    instance_name='episode',
    instance_scale=1,
    epochs=10,
    learning_rate=1e-3,
    random_latent=random_latent,
    latent_dim=LATENT_DIM ,
    recoding_dir=recoding_dir,
    gray_plot=True,
    generate_epoch=5,
    save_epoch=5
)